### Importación de librerias

In [56]:
import sqlite3
import pandas as pd
import sqlalchemy as db
import logging

### Creación de logging

In [57]:
# Creación de logging
logging.basicConfig(filename='./logs/result.log', encoding='utf-8',
                    level=logging.DEBUG, datefmt='%y-%m-%d %H:%M:%S',
                    format='%(asctime)s - %(levelname)s - %(message)s')

### Carga de datos

In [58]:
# Carga de los datos
try:
    medallas_olimpicas = pd.read_csv('http://winterolympicsmedals.com/medals.csv')
    logging.info('Archivo URL cargado exitosamente')
except Exception:
    medallas_olimpicas = pd.read_csv('./data/medals.csv')
    logging.warning(f'Archivo local cargado exitosamente *** Puede que no se encuentre actualizado ***')

### Verificación de los datos

In [59]:
# Verificación de los datos
medallas_olimpicas.sample(n=5)

,Year,City,Sport,Discipline,NOC,Event,Event gender,Medal
1751,1998,Nagano,Skating,Speed skating,NED,1000m,M,Silver
452,1960,Squaw Valley,Skating,Speed skating,URS,500m,M,Gold
2196,2006,Turin,Skating,Short Track S.,KOR,1500m,W,Gold
806,1976,Innsbruck,Luge,Luge,FRG,singles,M,Silver
2235,2006,Turin,Biathlon,Biathlon,RUS,15km,W,Gold


In [60]:
# Verificación de los tipos de datos
medallas_olimpicas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2311 entries, 0 to 2310
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Year          2311 non-null   int64 
 1   City          2311 non-null   object
 2   Sport         2311 non-null   object
 3   Discipline    2311 non-null   object
 4   NOC           2311 non-null   object
 5   Event         2311 non-null   object
 6   Event gender  2311 non-null   object
 7   Medal         2311 non-null   object
dtypes: int64(1), object(7)
memory usage: 144.6+ KB


In [61]:
# Verificación de todos los códigos existentes en el DataFrame
medallas_olimpicas['NOC'].unique()

array(['AUT', 'BEL', 'CAN', 'FIN', 'FRA', 'GBR', 'NOR', 'SUI', 'SWE',
       'USA', 'GER', 'TCH', 'HUN', 'ITA', 'FRG', 'NED', 'EUA', 'JPN',
       'POL', 'URS', 'PRK', 'GDR', 'ROU', 'ESP', 'LIE', 'BUL', 'YUG',
       'CHN', 'EUN', 'KOR', 'LUX', 'NZL', 'AUS', 'BLR', 'KAZ', 'RUS',
       'SLO', 'UKR', 'UZB', 'CZE', 'DEN', 'CRO', 'EST', 'LAT', 'SVK'],
      dtype=object)

In [62]:
# Verificación de que todos los años esten en formato númerico
medallas_olimpicas['Year'].unique()

array([1924, 1928, 1932, 1936, 1948, 1952, 1956, 1960, 1964, 1968, 1972,
       1976, 1980, 1984, 1988, 1992, 1994, 1998, 2002, 2006], dtype=int64)

In [63]:
# Verificación que todas las medallas hayan sido cargadas correctamente en el mismo formato
medallas_olimpicas['Medal'].unique()

array(['Silver', 'Gold', 'Bronze'], dtype=object)

### Filtrado de datos

In [64]:
# Filtrado de medallas de USA mayores a 1950 y de oro
medallas_oro_usa = medallas_olimpicas[(medallas_olimpicas['NOC']=='USA') &
                                      (medallas_olimpicas['Year']>=1950) &
                                      (medallas_olimpicas['Medal'] == 'Gold')]

### Verificación del filtrado de datos

In [65]:
# Verificación de los datos
medallas_oro_usa.sample(n=5)

,Year,City,Sport,Discipline,NOC,Event,Event gender,Medal
388,1956,Cortina d'Ampezzo,Skating,Figure skating,USA,individual,W,Gold
311,1952,Oslo,Skating,Figure skating,USA,individual,M,Gold
674,1968,Grenoble,Skating,Figure skating,USA,individual,W,Gold
893,1976,Innsbruck,Skating,Speed skating,USA,500m,W,Gold
2309,2006,Turin,Skiing,Snowboard,USA,Snowboard Cross,M,Gold


In [66]:
# Verificación que la longitud obtenida tenga coherencia
# Total de datos en USA
df_USA = medallas_olimpicas[(medallas_olimpicas['NOC']=='USA')]
# Total de datos en 1950 >
df_year = medallas_olimpicas[(medallas_olimpicas['Year']>=1950)]
# Total de datos con medallas de oro
df_oro = medallas_olimpicas[(medallas_olimpicas['Medal'] == 'Gold')]

# Logeo de eventos según la coherencia de los datos
# Verificamos que la cantidad de eventos de USA, Años y Medallas no supere al total de estos mismo
if (df_USA.shape[0] > medallas_oro_usa.shape[0] and df_year.shape[0] > medallas_oro_usa.shape[0] and 
    df_oro.shape[0] > medallas_oro_usa.shape[0]):
    logging.info('datos filtrados correctamente')
else:
    logging.warning('Los datos no mantienen coherencia ¡Por favor verifique el filtrado!')

### Conexión con la base de datos

In [67]:
try:
    # Conexión a la db
    conn = sqlite3.connect('olympics.db')
    # Creación del cursor
    cursor = conn.cursor()
    logging.info('Conexión realizada con exito')
except Exception:
    logging.error('¡No se pudo conextar a la base de datos!')

### Creación de tabla

In [68]:
try:
    # Buscamos nuestro script medallas.sql y leemos
    with open('./query/medallas.sql', 'r') as q:
        query = q.read()
        cursor.execute(query)
        conn.commit()
        logging.info('¡Creación de tabla exitosa!')
except Exception:
    logging.error('¡La tabla no pudo ser creada, verifique que exista la query!')

### Inserción de datos

In [69]:
try:
    # Transformación de los datos en tuplas dentro de una lista
    list_data = list(medallas_oro_usa.itertuples(index=False, name=None))
    # Se inserta dentro de nuestra tabla medals
    cursor.executemany("INSERT INTO medals VALUES (?, ?, ?, ?, ?, ?, ?, ?)", list_data)
    conn.commit()
    logging.info('Los datos fueron cargados exitosamente')
except Exception:
    logging.warning('Los datos no pudieron ser cargados')

### Lectura de datos de nuestra DB

In [70]:
try:
    # Buscamos nuestro script medallas.sql y leemos
    with open('./query/consulta_medallas.sql', 'r') as q:
        query = q.read()
        cursor.execute(query)
        # Se carga los datos obtenidos
        resultados = cursor.fetchall()
        # Se muestran los datos
        for result in resultados:
            print(result)
    logging.info('Los datos fueron mostrados exitosamente')
except Exception:
    logging.warning('¡Los datos no pudieron ser consultados, verifique que exista la query!')
# Cierre del cursor
conn.close()

(1952, 'Oslo', 'Skating', 'Figure skating', 'USA', 'individual', 'M', 'Gold')
(1952, 'Oslo', 'Skating', 'Speed skating', 'USA', '500m', 'M', 'Gold')
(1952, 'Oslo', 'Skiing', 'Alpine Skiing', 'USA', 'giant slalom', 'W', 'Gold')
(1952, 'Oslo', 'Skiing', 'Alpine Skiing', 'USA', 'slalom', 'W', 'Gold')
(1956, "Cortina d'Ampezzo", 'Skating', 'Figure skating', 'USA', 'individual', 'M', 'Gold')
(1956, "Cortina d'Ampezzo", 'Skating', 'Figure skating', 'USA', 'individual', 'W', 'Gold')
(1960, 'Squaw Valley', 'Ice Hockey', 'Ice Hockey', 'USA', 'ice hockey', 'M', 'Gold')
(1960, 'Squaw Valley', 'Skating', 'Figure skating', 'USA', 'individual', 'M', 'Gold')
(1960, 'Squaw Valley', 'Skating', 'Figure skating', 'USA', 'individual', 'W', 'Gold')
(1964, 'Innsbruck', 'Skating', 'Speed skating', 'USA', '500m', 'M', 'Gold')
(1968, 'Grenoble', 'Skating', 'Figure skating', 'USA', 'individual', 'W', 'Gold')
(1972, 'Sapporo', 'Skating', 'Speed skating', 'USA', '1500m', 'W', 'Gold')
(1972, 'Sapporo', 'Skating', 